In [7]:
# Import Standard Libraries
import os

# Import Third Party Libraries
import pytest

# Import Local Libraries

In [9]:
# Define Global Variables

# Define the path to /tests directory
TEST_DIR = os.path.join(os.getcwd(), '../tests')

# Define pytest verbosity level
VERBOSITY = '-q' # Quiet
# VERBOSITY = '-v' # Verbose
# VERBOSITY = '-vv' # More verbose
# VERBOSITY = '-vvv' # Even more verbose

# Define pytest traceback level
TRACEBACK = '--tb=auto' # Default
# TRACEBACK = '--tb=short' # Short
# TRACEBACK = '--tb=long' # Long
# TRACEBACK = '--tb=line' # One line
# TRACEBACK = '--tb=native' # Python standard
# TRACEBACK = '--tb=no' # No traceback

# Demonstration of STIX-D Clex Importer Tool

## Use Case

## Code Execution

## Project Design

## Code Interaction with Database

## Test Cases

In [8]:
# Run all tests in the test directory
pytest.main([TEST_DIR, VERBOSITY, TRACEBACK])


................................                                         [100%]
32 passed in 36.53s


<ExitCode.OK: 0>